In [1]:
#import marketdata.ext
import marketdata
import pickle
import math
import os
import pandas as pa
import numpy as np
import sys
import datetime as dt
from IPython.display import HTML
import pybacktest as pbt
#from strat.pbpsbth import persistent_locals2
from scipy import stats
import statsmodels.formula.api as sm
import patsy
import matplotlib.pyplot as plt
os.path.realpath('.')

ModuleNotFoundError: No module named 'cdecimal'

In [ ]:
sys.path

In [ ]:
#Calculating symbol earnings vola and working with data errors
def earnings_vola (symbol):
    try:
        price_b = marketdata.ext.load_metotron(symbol)
        price = np.log(price_b.sort())
    except:
        print ('symbol %s not found'%(symbol))
        return
#filtering stocks that have not enought data depth or low volume or last price < 3
    vol_check = (len(price_b) > 700) & (price_b['C'].tail(1) >= 3) & (pa.rolling_mean(price_b['V'], 20).tail(1) > 100000) & (pa.rolling_mean(price_b['V'], 20).tail(1) * price_b['C'].tail(1) >= 2000000)
    if vol_check:
        try:
            d = pickle.load(open('/home/mad/Appr/data_%s.pickle'%(symbol), 'rb'))
            df = h.to_df(d)
        except ValueError:
            print ('symbol %s foreign'%(symbol))
            return
        if len(df) > 0:
            if df['timetype'][0]=='After Close':
                price['VOLA'] = (price.shift(-1)['C']-price['C']).abs()
            else:
                price['VOLA'] = (price['C']-price.shift(+1)['C']).abs()
            x1 = pa.Series(sorted(df['Date_p']))
            px = price.reindex(x1, method='ffill')
            avg_vola = np.mean(px['VOLA'])
            return avg_vola
        else:
            print ('symbol %s have empty table'%(symbol))
            return
    else:    
        print ('symbol %s have low liquidity'%(symbol))
        return

In [ ]:
#Preparing comlete symbol list
l = os.listdir('/home/mad/Appr/'); symb_list = [v[5:-7] for v in l if v.startswith('data_') and v.endswith('.pickle')]

In [ ]:
#Filtering complete list with all exceptions and saving final list like 'symbol : vola'
#f2 = []
#for i in symb_list:
#    try:
#        f2.append((i, earnings_vola(i)))
#    except ValueError:
#        continue

In [ ]:
#Saving list to DF and then to csv
#f3 = pa.DataFrame(f2, columns = ['Symbol', 'Volat']).dropna()
#f3.to_csv('/home/mad/Appr/earnings_vola.csv')

In [ ]:
#Defining time exit (n-bar exit)
def nbarexit(ts, n):
    posstate = 0    
    count = 0
    Exit = pa.TimeSeries(index = [ts.index], dtype = bool)
    for i in xrange(len(ts)):
        if ts[i] == True:
            if posstate == 0:
                posstate = 1
                Exit[i] = False
                count += 1
            elif posstate == 1:
                if count < n:
                    Exit[i] = False
                    count += 1
                if count == n:
                    Exit[i] = True
                    count = 0
                    posstate = 0
        elif ts[i] == False:
            if posstate == 1: 
                if count < n:
                    Exit[i] = False
                    count += 1
                if count == n:            
                    Exit[i] = True
                    count = 0
                    posstate = 0    
            elif posstate == 0:
                    Exit[i] = False
    return Exit

In [ ]:
price_b = marketdata.ext.load_metotron('AAPL')
start = price_b.index.searchsorted(dt.datetime(1999, 12, 15))
ohlc = price_b.ix[start:]
O, H, L, C = ohlc.O, ohlc.H, ohlc.L, ohlc.C 
ohlc['HHV'] = pa.rolling_max(ohlc['H'].shift(1), 20)
ohlc['LLV'] = pa.rolling_min(ohlc['L'].shift(1), 20)
ohlc['buy_base'] = ohlc.C > ohlc.HHV
ohlc['sell'] = nbarexit(ohlc.buy_base, 15)
ohlc['buy'] = ohlc['buy_base'] & ~ ohlc['sell']
#ohlc.ix[35:75]

In [ ]:
def baseMOMO(ohlc):
    O, H, L, C = ohlc.O, ohlc.H, ohlc.L, ohlc.C 
    HHV = pa.rolling_max(ohlc['H'].shift(1), 20)
    LLV = pa.rolling_min(ohlc['L'].shift(1), 20)
    buy_base = C > HHV
    short = 0 #C < LLV
    sell = nbarexit(buy_base, 15)
    buy = buy_base & ~ sell
    cover = 0 #nbarexit(short, 5)
    res = dict(Buy=buy, Sell=sell, Short=short, Cover=cover)
    return res

In [ ]:
def MOMO(symbol):
    #loading data
    price_b = marketdata.ext.load_metotron(symbol)
    start = price_b.index.searchsorted(dt.datetime(1999, 12, 15))
    price_b = price_b.ix[start:]
    
    #getting buy, sell, short, cover signals vectors
    res = baseMOMO(price_b)
    
    #specifying pure OHLC dataframe
    ohlc = pa.DataFrame(price_b, columns=('O', 'H', 'L', 'C'))
    
    #specifying trade moment
    #buyprice = shortprice = ohlc.O
    #sellprice = coverprice = ohlc.C
    
    #getting backtest object
    bct = pbt.Backtest(dict(ohlc=ohlc, buy=res['Buy'], sell=res['Sell'],short=res['Short'], cover=res['Cover']), name = 'Base MOMO str')
    
    #getting perfomance report and profit factor
    perf = bct.report
    pf = perf['performance']['PF']
    
    return pf, bct

In [ ]:
#loading filtered list of stocks & reindexing
slist = pa.DataFrame.from_csv('/home/mad/Appr/earnings_vola.csv')
symblist = pa.DataFrame(slist.reset_index(drop=True))
symblist.columns = ['Symbol', 'Volat']
symblist['PF'] = ''
symblist['Trades'] = ''
resdf = symblist

In [ ]:
#iterating all symbols and getting 'PF' statistic
for i in xrange(len(symblist)):
    try:
        s = symblist.Symbol[i]
        pf, bct = MOMO(s)
        trades = bct.stats.trades
        if trades > 50:
            resdf['PF'][i] = pf
            resdf['Trades'][i] = trades
            print s
        else:
            print ('symbol %s have not enought trades'%(symblist.Symbol[i]))            
    except ValueError:
        print ('symbol %s have empty data'%(symblist.Symbol[i]))

In [ ]:
#filtering empty 'string' PF
#resd


###
#f = resdf.ix[resdf.PF != '']

#converting dtypes of series from string to float
resdf = resdf.convert_objects(convert_numeric=True)

#Saving resulting DF to csv
resdf.to_csv('/home/mad/Appr/vola_pf.csv')

In [ ]:
resdf1 = pa.DataFrame.from_csv('/home/mad/Appr/vola_pf.csv')
resdf1 = resdf1.dropna()


In [ ]:
resdf_sorted = resdf1.sort(columns='Volat', ascending=True)
top100 = resdf_sorted.head(300)
top100.head(20)

In [ ]:
f = 'PF ~ Volat'
y,X = patsy.dmatrices(f, resdf1, return_type='dataframe')
#print sm.OLS(y, X).fit().summary()
model = sm.ols('PF ~ Volat', top100)
fitted = model.fit()
print fitted.summary()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(top100["Volat"], top100["PF"], 'ro')
plt.plot(top100["Volat"], fitted.fittedvalues, 'b')
plt.legend(['Data', 'Fitted model'])
#plt.ylim(0, 10)
#plt.xlim(-2, 12)
plt.xlabel('Earnings volatility')
plt.ylabel('Profit Factor')
plt.title('Volatility to Earnings Lin Regr')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.hist(fitted.norm_resid())
plt.ylabel('Count')
plt.xlabel('Normalized residuals')
plt.title('Residuals histogramm')
plt.show()

In [ ]:
#from statsmodels.graphics.regressionplots import *
#fig, ax = plt.subplots(figsize=(12,8))
#influence_plot(fitted)
#plot_leverage_resid2(fitted)

In [ ]:
top100.ix[top100.Symbol == 'PGR']
#resdf1.ix[resdf1.Symbol == 'GRPN']

In [ ]:
pf, bct = MOMO('GRPN')
pf

In [ ]:
trades = bct.stats.trades
trades

In [ ]:
resdf_sorted.ix[resdf_sorted.Symbol == 'NVAX']

In [ ]:
 d = pickle.load(open('/home/mad/Appr/data_%s.pickle'%('MFC'), 'rb'))

In [ ]:
bct.plot_equity()